In [ ]:
!pip install autogluon.timeseries

In [ ]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
data = pd.read_csv('three input.csv')
train_data = data[['timestamp','Temp_Box','Temp','item_id']]
train_data.tail(240)

In [ ]:
train_data['timestamp'] = train_data['timestamp'].astype('datetime64')
train_data_partial_per_item = []
train_data_item_id = pd.DataFrame(train_data).groupby('item_id')
for i,j in train_data_item_id:
    train_data_partial_per_item.append(pd.DataFrame(j[:72]).set_index(['item_id','timestamp']))
    #print(train_data_partial_per_item)
train_data_partial = TimeSeriesDataFrame(pd.concat(train_data_partial_per_item))

In [ ]:
train_data_partial

In [ ]:
import autogluon.core as ag
while len(train_data_partial) < len(train_data):
  if len(train_data_partial.loc[1]) < 49:
    pred_len = 1
  elif len(train_data_partial.loc[1]) >= 49 and len(train_data_partial.loc[1]) < 72:
    pred_len = 23
  elif len(train_data_partial.loc[1]) >= 72:
    pred_len =  24
  elif len(train_data_partial.loc[1]) == 240:
    break

  predictor = TimeSeriesPredictor(
      prediction_length= pred_len,
      target="Temp",
      known_covariates_names=["Temp_Box"],
      eval_metric="RMSE",
      ignore_time_index = True
      ).fit(train_data_partial,
            #hyperparameters = {"DeepAR" : {"hidden_size": ag.space.Int(20, 100),}},
            #hyperparameter_tune_kwargs="auto",
            hyperparameters = {"DeepAR" : {}}
        )

  known_covariates_per_item = []
  for i,j in train_data_item_id:
    known_covariate = pd.DataFrame(j[len(train_data_partial.loc[1]): len(train_data_partial.loc[1])+pred_len]).set_index(['item_id','timestamp'])
    #known_covariate['Temp_Easy'] = known_covariate['Temp_Box']
    known_covariates_per_item.append(pd.DataFrame(known_covariate[['Temp_Box']]))
  known_covariates = TimeSeriesDataFrame(pd.concat(known_covariates_per_item))
  predict_data = predictor.predict(train_data_partial, known_covariates = known_covariates)
  new_train_data_per_item = []
  for i in range(len(known_covariates_per_item)):
    predicted_data_chunk = pd.DataFrame(predict_data[['mean']].values[i*len(known_covariates_per_item[i]):(i+1)*len(known_covariates_per_item[i])], index = pd.DataFrame(known_covariates_per_item[i]).index, columns = ["Temp"])
    together =  pd.concat([pd.DataFrame(known_covariates_per_item[i]), predicted_data_chunk], axis = 1)
    if len(train_data_partial.loc[1]) == 72:
      together = pd.concat([train_data_partial_per_item[i], together], axis = 0)
    else:
      idx = together.index[0][0]
      together = pd.concat([train_data_partial.loc[together.index[0][0]], together.loc[together.index[0][0]]], axis = 0)
      together.reset_index(inplace = True)
      together['item_id'] = idx
      together = together.set_index(['item_id','timestamp'])
    new_train_data_per_item.append(together)

  train_data_partial = TimeSeriesDataFrame(pd.concat(new_train_data_per_item))
train_data_partial